In [2]:
import polars as pl
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath('')))

from libraries.FaceDatasetBuilder import FaceDatasetBuilder
from libraries.client_stashapp import get_stashapp_client, StashAppClient

def get_stashdb_performer(performer):
    """Extract StashDB ID and name from performer data"""
    for stash_id in performer["stashapp_performers_stash_ids"]:
        if stash_id["endpoint"] == "https://stashdb.org/graphql":
            return {
                'stash_id': stash_id["stash_id"],
                'name': performer["stashapp_performers_name"]
            }
    return None

# Process a scene
def process_scene(scene):
    # Get performers with their StashDB IDs
    performers = []
    for performer in scene["stashapp_performers"]:
        stashdb_info = get_stashdb_performer(performer)
        if stashdb_info:
            performers.append(f"{stashdb_info['stash_id']} - {stashdb_info['name']}")
    
    # Process the scene
    if performers:
        return builder.process_scene(
            video_path=scene["stashapp_primary_file_path"],
            scene_id=scene["stashapp_stashdb_id"],
            performers=performers
        )
    return 0

# Initialize builder
builder = FaceDatasetBuilder()

stash = get_stashapp_client()
stash_client = StashAppClient()

dUsing stash (v0.27.2-37-g0621d871) endpoint at http://localhost:6969/graphql
dPersisting Connection to Stash with ApiKey...
dUsing stash (v0.27.2-37-g0621d871) endpoint at http://localhost:6969/graphql
dPersisting Connection to Stash with ApiKey...


In [14]:
all_performers = stash_client.get_performers()
performer = all_performers.filter(pl.col("stashapp_name").str.contains("Alexis Crystal")).to_dicts()[0]

In [15]:
sample_scenes = stash_client.find_scenes_by_performers([performer["stashapp_id"]])

In [16]:
sample_scene = sample_scenes.to_dicts()[1]
sample_scene

{'stashapp_id': 656,
 'stashapp_title': 'Crystal Clear',
 'stashapp_details': 'Alexis Crystal loves having her feet played with and Renato is happy to do that for her. He licks her toes and licks her pussy before sliding his hard cock inside of her. She moans in pleasure and after awhile, he cums all over her feet!',
 'stashapp_date': datetime.date(2018, 11, 6),
 'stashapp_urls': ['https://www.21naturals.com/en/video/Crystal-Clear/134308'],
 'stashapp_created_at': datetime.datetime(2023, 1, 25, 10, 7, 22),
 'stashapp_updated_at': datetime.datetime(2025, 2, 15, 4, 9, 8),
 'stashapp_performers': [{'stashapp_performers_id': 139,
   'stashapp_performers_name': 'Alexis Crystal',
   'stashapp_performers_disambiguation': '',
   'stashapp_performers_alias_list': ['Anouk',
    'Carrie',
    'Dominika',
    'Emilia',
    'Kittycat',
    'Liz',
    'Mara',
    'Sindy'],
   'stashapp_performers_gender': 'FEMALE',
   'stashapp_performers_favorite': True,
   'stashapp_performers_stash_ids': [{'endpo

In [17]:
# Process your sample scene
faces_extracted = process_scene(sample_scene)
print(f"Extracted {faces_extracted} faces")

# Verify directories exist
import os
base_dir = "H:\\Faces\\dataset"
print("\nDirectory structure:")
for root, dirs, files in os.walk(base_dir):
    level = root.replace(base_dir, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    if level < 2:  # Only show first two levels
        for d in dirs:
            print(f"{indent}    {d}/")


Processing scene: ca1bc690-a26a-402b-a71b-325e97168478
Creating performer directory: H:\Faces\dataset\unverified\ca1bc690-a26a-402b-a71b-325e97168478\d5061b46-796b-4204-8e4f-cff4569fdea6 - Alexis Crystal
Creating performer directory: H:\Faces\dataset\unverified\ca1bc690-a26a-402b-a71b-325e97168478\f55e5bca-7e77-4787-a3e3-1ad03ef2d189 - Renato
Extracted 66 faces

Directory structure:
dataset/
    temp/
    unverified/
    verified/
    temp/
    unverified/
        25125eb0-2d9e-43a1-8497-d6a8f12c0522/
        275e0955-4587-4012-bdef-680bffe9dca1/
        ca1bc690-a26a-402b-a71b-325e97168478/
        25125eb0-2d9e-43a1-8497-d6a8f12c0522/
            df3febe9-2c0d-4cca-a824-fa8b3340bc56 - Eva de Vil/
            rejected/
        275e0955-4587-4012-bdef-680bffe9dca1/
            df3febe9-2c0d-4cca-a824-fa8b3340bc56 - Eva de Vil/
            rejected/
        ca1bc690-a26a-402b-a71b-325e97168478/
            d5061b46-796b-4204-8e4f-cff4569fdea6 - Alexis Crystal/
            f55e5bca-7e77

In [ ]:
# After manually moving faces to correct performer directories in Windows Explorer
scene_stashdb_id = sample_scene["stashapp_stashdb_id"]
builder.verify_scene(scene_stashdb_id)

# Check how many faces we have for each performer
face_counts = builder.get_performer_face_count()
for performer, count in sorted(face_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{performer}: {count} faces")

Scene ca1bc690-a26a-402b-a71b-325e97168478 verification completed
df3febe9-2c0d-4cca-a824-fa8b3340bc56 - Eva de Vil: 78 faces
d5061b46-796b-4204-8e4f-cff4569fdea6 - Alexis Crystal: 41 faces
f55e5bca-7e77-4787-a3e3-1ad03ef2d189 - Renato: 22 faces
Eva de Vil has 78 verified faces
